In [1]:
import os

In [2]:
%pwd

'/Users/liangzhang/Documents/ai/dl-tutorial/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/liangzhang/Documents/ai/dl-tutorial'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/liang3030/dl-tutorial.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="liang3030"
os.environ["MLFLOW_TRACKING_PASSWORD"]="XXX" # replace with your DAGsHub API key

In [6]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
		path_of_model: Path
		training_data: Path
		all_params: dict
		mlflow_uri: str
		params_image_size: list
		params_batch_size: int

In [10]:
from template_dl_tutorial.constants import *
from template_dl_tutorial.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
	def __init__(
		self,
		config_filePath = CONFIG_FILE_PATH,
		params_filePath = PARAMS_FILE_PATH
	):
		self.config = read_yaml(config_filePath)
		self.params = read_yaml(params_filePath)

		create_directories([self.config.artifacts_root])


	def get_evaluation_config(self) -> EvaluationConfig:
		eval_config = EvaluationConfig(
			path_of_model = "artifacts/training/model.h5",
			training_data="artifacts/data_ingestion/kidney-ct-scan-image",
			mlflow_uri='https://dagshub.com/liang3030/dl-tutorial.mlflow',
			all_params = self.params,
			params_image_size = self.params.IMAGE_SIZE,
			params_batch_size = self.params.BATCH_SIZE
		)

		return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
	def __init__(self, config: EvaluationConfig):
		self.config = config
	def _valid_generator(self):
		datagenerator_kwargs = dict(
			rescale=1./255,
			validation_split=0.30,
		)

		dataflow_kwargs = dict(
			target_size=self.config.params_image_size[:-1],
			batch_size=self.config.params_batch_size,
			interpolation="bilinear",
		)

		valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

		self.valid_generator = valid_datagenerator.flow_from_directory(
			directory=self.config.training_data,
			subset="validation",
			shuffle=False,
			**dataflow_kwargs,
		)

	@staticmethod
	def load_model(path: Path) -> tf.keras.models.Model:
		return tf.keras.models.load_model(path)
	
	def evaluation(self):
		self.model = self.load_model(self.config.path_of_model)
		self._valid_generator()
		self.score = self.model.evaluate(self.valid_generator)
		self.save_score()

	def save_score(self):
		scores = {"loss": self.score[0], "accuracy": self.score[1]}
		save_json(path=Path("scores.json"), data=scores)

	def log_into_mlflow(self):
		mlflow.set_registry_uri(self.config.mlflow_uri)
		tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

		with mlflow.start_run():
			mlflow.log_params(self.config.all_params)
			mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

			# TODO: bug in log_model
			# if tracking_url_type_store != "file":
			# 	mlflow.keras.log_model(self.model, "model")
			# else:
			# 	mlflow.keras.log_model(self.model, "model")


In [15]:
try:
	config = ConfigurationManager()
	eval_config = config.get_evaluation_config()
	evaluation = Evaluation(eval_config)
	evaluation.evaluation()
	evaluation.log_into_mlflow()
except Exception as e:
	raise e

[2024-06-01 17:32:21,682]: INFO: common: yaml file: config/config.yaml loaded successfully
[2024-06-01 17:32:21,685]: INFO: common: yaml file: params.yaml loaded successfully
[2024-06-01 17:32:21,686]: INFO: common: Created directory at artifacts
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 19.3878 - accuracy: 0.4820
[2024-06-01 17:32:34,803]: INFO: common: Saved json file at scores.json
